In [1]:
import psycopg2
import json
import io
import csv
from dateutil.parser import isoparse

In [2]:
connection = psycopg2.connect(
    host="localhost",
    port=6021,
    database="eksi",
    user="postgres",
    password="docker")

In [3]:
cursor = connection.cursor()

In [4]:
from sql_queries import create_table_queries, drop_table_queries

In [5]:
for query in drop_table_queries:
    cursor.execute(query)
    connection.commit()

In [6]:
for query in create_table_queries:
    cursor.execute(query)
    connection.commit()

In [7]:
with open('./data/sample_entries.json', 'r', encoding='utf-8') as jsonfile:
    for item in jsonfile:
        entry = json.loads(item)
        new_json = {
            "entry_id" : entry['Id'],
            "author_nick" : entry['Author']['Nick'],
            "author_id" : entry['Author']['Id'],
            "created" : entry['Created'],
            "last_updated" :entry['LastUpdated'],
            "is_favorite" : entry['IsFavorite'],
            "favorite_count" : entry['FavoriteCount'],
            "hidden" : entry['Hidden'],
            "active" : entry['Active'],
            "comment_count" : entry['CommentCount'],
            "comment_summary" : entry['CommentSummary'],
            "avatar_url" : entry['AvatarUrl'],
            "topic_id" : entry['TopicId'],
            "topic_title" : entry['TopicTitle'],
            "content" : entry['Content'],
        }
        try:
            loaded_json = json.loads(json.dumps(new_json))
            tab_splited_values = io.StringIO("\t".join(map(str,loaded_json.values())))
            cursor.copy_from(tab_splited_values, 'staging_entries', sep='\t', null="None")
            connection.commit()
        except (Exception, psycopg2.DatabaseError) as e:
            print(e)


In [8]:
with open('./data/sample_authors_detail.json', 'r', encoding='utf-8') as jsonfile:
    for item in jsonfile:
        author = json.loads(item)

        db_badges = None
        if 'Badges' in author:
            db_badges = ",".join([item["Name"] for item in author["Badges"]])

        karma_name, karma_value = None, -1
        if  author['UserInfo']['Karma']:
            karma_name = author['UserInfo']['Karma']['Name']
            karma_value =  author['UserInfo']['Karma']['Value']

        new_json = {
            'author_id' : author['UserId'],
            'author_nick' : author['UserInfo']['UserIdentifier']['Nick'],
            'twitter_screen_name' : author['UserInfo']['TwitterScreenName'],
            'facbook_profile_url' : author['UserInfo']['FacebookProfileUrl'],
            'facebook_screen_name' : author['UserInfo']['FacebookScreenName'],
            'instagram_screen_name' : author['UserInfo']['InstagramScreenName'],
            'instagram_profile_url' : author['UserInfo']['InstagramProfileUrl'],
            'karma_name' : karma_name,
            'karma_value' : karma_value,
            'entry_count' : author['UserInfo']['EntryCounts']['Total'],
            'last_entry_date' : author['UserInfo']['LastEntryDate'],
            'is_corporate' : author['UserInfo']['IsCorporate'],
            'is_deactivated' : author['UserInfo']['IsDeactivated'],
            'is_caylak' : author['UserInfo']['IsCaylak'],
            'is_cursed' : author['UserInfo']['IsCursed'],
            'follower_count' : author['FollowerCount'],
            'followings_count' : author['FollowingsCount'],
            'picture' : author['Picture'],
            'has_entry_on_seyler' : author['HasEntryUsedOnSeyler'],
            'badges' :  db_badges,
            'favorite_entries': '' if 'FavoriteEntries' not in author else author['FavoriteEntries'],
            'favorited_dntries': '' if 'FavoritedEntries' not in author else author['FavoritedEntries']
        }
        try:
            loaded_json = json.loads(json.dumps(new_json))
            tab_splited_values = io.StringIO("\t".join(map(str,loaded_json.values())))
            cursor.copy_from(tab_splited_values, 'staging_author_details', sep='\t', null="None")
            connection.commit()
        except (Exception, psycopg2.DatabaseError) as e:
            print(e)


In [9]:
with open('./data/sample_topics.json', 'r', encoding='utf-8') as jsonfile:
    for item in jsonfile:
        topics = json.loads(item)
        new_json = {
          'topic_id' : topics['TopicId'],
          'topic_title' : topics['TopicTitle'],
          'entry_count' : topics['EntryCount'],
        }
        try:
            loaded_json = json.loads(json.dumps(new_json))
            tab_splited_values = io.StringIO("\t".join(map(str,loaded_json.values())))
            cursor.copy_from(tab_splited_values, 'staging_topics', sep='\t', null="None")
            connection.commit()
        except (Exception, psycopg2.DatabaseError) as e:
            print(e)


In [10]:
with open('./data/sample_authors.json', 'r', encoding='utf-8') as jsonfile:
    for item in jsonfile:
        author = json.loads(item)
        new_json = {
            'author_id' : author['Id'],
            'author_nick' : author['Nick'],
        }
        try:
            loaded_json = json.loads(json.dumps(new_json))
            tab_splited_values = io.StringIO("\t".join(map(str,loaded_json.values())))
            cursor.copy_from(tab_splited_values, 'staging_authors', sep='\t', null="None")
            connection.commit()
        except (Exception, psycopg2.DatabaseError) as e:
            print(e)


In [ ]:
# cursor.execute("ROLLBACK")
# connection.commit()

In [12]:
# cursor.execute("select * from staging_entries;")
# cursor.fetchall()